In [1]:
import nltk
from tqdm.notebook import tqdm

In [2]:
f = open("tags-processed-unique.txt", "r")
lines = f.readlines()
f.close()

In [3]:
lines = lines[0].split(", ")

In [4]:
len(lines)

1042

In [5]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ragha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ragha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
from nltk.corpus import wordnet as wn

In [7]:
def get_synonyms(word):
    print(word)
    return list(map(lambda x: x.name(), wn.synsets(word)[0].lemmas()))

In [8]:
get_synonyms(lines[0])

drunken speech


IndexError: list index out of range

In [9]:
wn.synsets(lines[0].split()[0])

[Synset('bibulous.s.01')]

In [11]:
# # Example list of words
# word_list = lines[:]

# # Remove synonyms from word list
# new_word_list = []
# for word in tqdm(word_list):
#     synonyms = set()
#     for syn in wn.synsets(word):
#         for lemma in syn.lemmas():
#             synonyms.add(lemma.name())
#     if not set(word_list) & synonyms:
#         new_word_list.append(word)
#     else:
#         print(word, "is a synonym of", list(set(word_list) & synonyms)[0])

# # Print new word list without synonyms
# print(new_word_list[:10])

In [10]:
def remove_synonyms(words):
    new_word_list = []
    for word in words:
        synsets = wn.synsets(word)
        synonyms = set()
        for synset in synsets:
            for lemma in synset.lemmas():
                if lemma.name() != word:
                    synonyms.add(lemma.name())
        if not synonyms:
            new_word_list.append(word)
        else:
            has_self_synonym = False
            for synonym in synonyms:
                if synonym != word and synonym in words:
                    has_self_synonym = True
                    break
            if not has_self_synonym:
                new_word_list.append(word)
    return new_word_list
new_tags = remove_synonyms(lines)
len(new_tags)

756

In [12]:
from langdetect import detect
from googletrans import Translator

translator = Translator()

def is_english(word):
    try:
        lang = detect(word)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

def remove_non_english(words):
    english_words = []
    for word in tqdm(words):
        if is_english(word):
            english_words.append(word)
        else:
            try:
                translation = translator.translate(word, dest='en')
                english_words.append(translation.text)
            except:
                pass
    return english_words

In [15]:
new_tags = remove_non_english(new_tags)

  0%|          | 0/756 [00:00<?, ?it/s]

In [16]:
new_tags = remove_synonyms(new_tags)

In [17]:
len(new_tags)

235

In [19]:
# write the result
f = open("tags-processed-final.txt", "w")
for tag in new_tags:
    f.write(tag + ",")
f.close()